# Security Agent

In [12]:
dt2ts = lambda dt: float(calendar.timegm(dt.timetuple()))

In [14]:
ftp_st_dt = datetime.datetime(2017, 7, 4, 9, 20)
ftp_et_dt = datetime.datetime(2017, 7, 4, 10, 20)

ftp_st = dt2ts(ftp_st_dt)
ftp_et = dt2ts(ftp_et_dt)

ssh_st_dt = datetime.datetime(2017, 7, 4, 14, 0)
ssh_et_dt = datetime.datetime(2017, 7, 4, 15, 0)

ssh_st = dt2ts(ssh_st_dt)
ssh_et = dt2ts(ssh_et_dt)

print ftp_st
print ftp_et
print ssh_st
print ssh_et

1499160000.0
1499163600.0
1499176800.0
1499180400.0


# NetFlow

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder

# Loading NetFlow Data

In [2]:
path = './data/netflow_data/'

#### Load all days

In [3]:
# Load all days
files = os.listdir(path)
data = {}
for f in files:
    data[f.replace('.csv', '')] =  pd.read_csv(path + f)

/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3,6,20,21,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21,85) have mixed types. Sp

In [4]:
# Filenames
data.keys()

['Monday-WorkingHours.pcap_ISCX',
 'Tuesday-WorkingHours.pcap_ISCX',
 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX',
 'Friday-WorkingHours-Morning.pcap_ISCX',
 'Wednesday-workingHours.pcap_ISCX',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX',
 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX']

#### Fix column headers

In [5]:
for k, d in data.iteritems():
    d.columns = d.columns.str.lower().str.strip().str.replace(
        ' ', '_').str.replace('/s', '').str.replace('/', '_').str.replace('.', '_')

#### Features

In [6]:
feat = set()
for k, v in data.iteritems():
    object_cols = list(v.dtypes[v.dtypes == 'object'].keys())
    all_cols = list(v.columns)
    feat.update(list(filter(lambda x: x not in object_cols, all_cols)))
# These features have type object so keeping them to convert to float64
feat.add('flow_bytes') 
feat.add('flow_packets')
feat = list(feat)

#### Merging Data

In [7]:
# has NaNs deal later
days_to_ignore = ['Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX']

In [8]:
cols = feat + ['label'] # Columns to keep from data
data_df = pd.DataFrame(columns=cols)
for k, d in data.iteritems():
    if k not in days_to_ignore:
        data_df = pd.concat((data_df, d[cols]), ignore_index=True)
data_df

flow_iat_mean flow_duration  packet_length_variance protocol  \
0             4.000000             4                0.000000        6   
1             1.000000             1                0.000000        6   
2             1.000000             1                0.000000        6   
3             1.000000             1                0.000000        6   
4             3.000000             3                0.000000        6   
5             1.000000             1                0.000000        6   
6             1.000000             1                0.000000        6   
7             1.000000             1                0.000000        6   
8            60.900000           609            11562.151515        6   
9            73.250000           879           300131.450549        6   
10           82.857143          1160           469182.516667        6   
11           52.400000           524           476021.878788        6   
12            6.000000             6                0.000000        6   
13           79.928571          1119           475055.266667        6   
14          735.120000         18378           490692.994302        6   
15           82.200000           822             9550.878788        6   
16           73.000000           876           276653.208791        6   
17          100.900000          1009           458562.969697        6   
18           61.600000           616            11562.151515        6   
19            5.000000             5                0.000000        6   
20           78.833333           946           300117.032967        6   
21           66.214286           927           475055.266667        6   
22           69.500000           973           460376.866667        6   
23           95.300000           953           479105.060606        6   
24           56.923077           740           531153.123810        6   
25         3349.260870         77033           197922.333333        6   
26           46.000000            46                0.000000        6   
27           74.133333          1112           448369.500000        6   
28           65.500000           655           476021.878788        6   
29         6773.833333         40643             4393.357143        6   
...                ...           ...                     ...      ...   
2660347  119835.000000        119835             1121.333333       17   
2660348   52121.000000        156363              529.200000       17   
2660349  155149.000000        155149             3008.333333       17   
2660350  422735.280000      10568382           267876.430200        6   
2660351   40113.333330        120340             1009.200000       17   
2660352      52.000000            52                0.000000        6   
2660353   33265.000000        365915            24781.769230        6   
2660354     223.500000           447              156.250000        6   
2660355   16583.000000         16583                0.000000       17   
2660356   33231.818180        365550            25492.025640        6   
2660357  336425.200000       3364252             4636.083333        6   
2660358      11.000000            11                0.000000        6   
2660359  116016.000000        116016                0.000000        6   
2660360   23228.000000         23228                0.000000       17   
2660361     667.000000           667                0.000000        6   
2660362  121289.000000        121289                0.000000        6   
2660363   14989.535710       1259121            57449.784950        6   
2660364    6663.454545         73298            35399.641030        6   
2660365      41.666667           125                0.000000       17   
2660366    9158.037037        247267            10256.743840        6   
2660367    6569.363636         72263            32033.692310        6   
2660368      87.000000           261               76.800000       17   
2660369   15102.243900       1238384            58391.238670       

#### Fix dtypes

In [9]:
data_df[feat] = data_df[feat].astype(pd.np.float64)

# Preprocessing

#### Imputing NaNs

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print data_df.isnull().sum()

flow_iat_mean                     0
flow_duration                     0
packet_length_variance            0
protocol                          0
flow_bytes                     1338
total_backward_packets            0
idle_mean                         0
psh_flag_count                    0
subflow_fwd_bytes                 0
bwd_avg_bytes_bulk                0
bwd_iat_total                     0
packet_length_mean                0
avg_bwd_segment_size              0
active_mean                       0
bwd_avg_bulk_rate                 0
act_data_pkt_fwd                  0
fwd_packets                       0
packet_length_std                 0
idle_min                          0
average_packet_size               0
syn_flag_count                    0
fwd_iat_total                     0
rst_flag_count                    0
bwd_iat_max                       0
fwd_iat_std                       0
cwe_flag_count                    0
fwd_packet_length_std             0
down_up_ratio               

In [11]:
data_df['flow_bytes'][data_df['flow_bytes'].isnull()] = 0

/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
print 'No NaNs' if sum(data_df.isnull().any()) == 0 else 'Found NaNs'

No NaNs


#### Removing Infs

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (data_df == float('Inf')).sum()

flow_iat_mean                     0
flow_duration                     0
packet_length_variance            0
protocol                          0
flow_bytes                     1394
total_backward_packets            0
idle_mean                         0
psh_flag_count                    0
subflow_fwd_bytes                 0
bwd_avg_bytes_bulk                0
bwd_iat_total                     0
packet_length_mean                0
avg_bwd_segment_size              0
active_mean                       0
bwd_avg_bulk_rate                 0
act_data_pkt_fwd                  0
fwd_packets                       0
packet_length_std                 0
idle_min                          0
average_packet_size               0
syn_flag_count                    0
fwd_iat_total                     0
rst_flag_count                    0
bwd_iat_max                       0
fwd_iat_std                       0
cwe_flag_count                    0
fwd_packet_length_std             0
down_up_ratio               

In [14]:
data_df = data_df.replace([np.inf, -np.inf], np.nan).dropna()

## Writing data

In [15]:
data_df.to_csv('./data/netflow.csv', index=False)

## Loading processed

In [7]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder

In [8]:
data_df = pd.read_csv('./data/netflow.csv')
feat = list(data_df.columns)
feat.remove('label')

#### Ignoring encoding for now

#### One hot encode categorical

In [9]:
# categ_feat = ['source_port', 'destination_port', 'protocol']

# data_df = data.itervalues().next()
# data_df = pd.get_dummies(data_df, prefix=['sp', 'dp', 'pr'], columns=categ, sparse=True)
# data_df

In [10]:
# enc = OneHotEncoder()
# enc.fit(data_df)
# transformed = enc.transform(data_df)

#### Scaling data

In [11]:
from sklearn.preprocessing import *
import pickle

#### Dropping features having very close max and min values to avoid NaNs on scaling

In [12]:
diff = data_df[feat].max(axis=0) - data_df[feat].min(axis=0)
eps = 0.00000000001

feat = list(data_df[feat].columns[~(diff.abs() < eps)])

In [13]:
# Considering all features numerical for now  Note: source_port, destination_port, protocol are
num_feat = feat

# Using MinMaxScaler
min_max = MinMaxScaler()
# Fit scaler
min_max.fit(data_df[num_feat].as_matrix())
# Save scaler using pickle
pickle.dump(min_max, open("min_max_data.p", "wb"))
# Transform data
data_df[num_feat] = pd.DataFrame(min_max.transform(
    data_df[num_feat].as_matrix()), columns=num_feat)

#### Encoding labels

In [14]:
le = LabelEncoder()
le.fit(data_df['label'].as_matrix())
pickle.dump(le, open("label_encoder.p", "wb"))

data_df['label'] = pd.DataFrame(le.transform(data_df['label'].as_matrix()), columns=['label'])

#### Splitting up data

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X = data_df[feat].as_matrix()
y = data_df['label'].as_matrix()

In [17]:
# Splitting Train Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, stratify=y)

/home/ahmed/Programs/anaconda3/envs/rl/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [19]:
# Splitting Train Validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.8, stratify=y_train)

/home/ahmed/Programs/anaconda3/envs/rl/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [20]:
print (X_train.shape)
print (y_train.shape)
print (X_val.shape)
print (y_val.shape)
print (X_test.shape)
print (y_test.shape)

(1700892, 72)
(1700892,)
(425224, 72)
(425224,)
(531529, 72)
(531529,)


In [29]:
labels = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
labels

{'BENIGN': 0,
 'Bot': 1,
 'DDoS': 2,
 'DoS GoldenEye': 3,
 'DoS Hulk': 4,
 'DoS Slowhttptest': 5,
 'DoS slowloris': 6,
 'FTP-Patator': 7,
 'Heartbleed': 8,
 'Infiltration': 9,
 'PortScan': 10,
 'SSH-Patator': 11}

In [11]:
print 'Benign flows in Training data:', sum(y_train == 1)
print 'Malicious flows in Training data:', sum(y_train == 0)
print 'Benign flows in Validation data:', sum(y_val == 1)
print 'Malicious flows in Validation data:', sum(y_val == 0)

for k, v in labels.iteritems():
    print 
    

Benign flows in Training data: 56149
Malicious flows in Training data: 77800
Benign flows in Validation data: 14037
Malicious flows in Validation data: 19451


# Building the Model

In [2]:
import tensorflow as tf
import keras as kr

Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop

## Building the model

In [15]:
# Training parameters
batch_size = 256 # 64 128 256
epochs = 600
feat = X_train.shape[1]
alpha = 0.01

In [16]:
# Keras sequential model
model = Sequential()
model.add(Dense(1024, input_shape=(feat, ), activation='relu', kernel_initializer='he_normal'))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='VarianceScaling'))
adam = Adam(lr=alpha, decay=0.0005)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              679936    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
__________

In [17]:
%%time
checkpoint = ModelCheckpoint(filepath='./saved_model/24-Dec-Model-1.hdf5', verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', batch_size=batch_size)
res = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[checkpoint, tensorboard]) 

Train on 133949 samples, validate on 33488 samples
Epoch 1/600
133949/133949 [==============================] - 6s 42us/step - loss: 0.2868 - acc: 0.8586 - val_loss: 0.2227 - val_acc: 0.8922
Epoch 2/600
133949/133949 [==============================] - 5s 40us/step - loss: 0.2022 - acc: 0.8989 - val_loss: 0.2005 - val_acc: 0.9044
Epoch 3/600
133949/133949 [==============================] - 5s 40us/step - loss: 0.1780 - acc: 0.9119 - val_loss: 0.1902 - val_acc: 0.9042
Epoch 4/600
133949/133949 [==============================] - 5s 40us/step - loss: 0.1622 - acc: 0.9199 - val_loss: 0.1883 - val_acc: 0.9096
Epoch 5/600
133949/133949 [==============================] - 5s 40us/step - loss: 0.1500 - acc: 0.9273 - val_loss: 0.1871 - val_acc: 0.9130
Epoch 6/600
133949/133949 [==============================] - 5s 39us/step - loss: 0.1384 - acc: 0.9328 - val_loss: 0.1837 - val_acc: 0.9150
Epoch 7/600
133949/133949 [==============================] - 5s 39us/step - loss: 0.1280 - acc: 0.9392 - val_

KeyboardInterrupt: 